In [7]:
from scipy.io.wavfile import write, read
import matplotlib.pyplot as plt
import os, sys
import numpy as np
import math 

In [8]:
def artCont(audio, path_base):
    fs, data_audio=read(audio)
    size_frameS=0.04*float(fs)
    size_stepS=0.02*float(fs)
    nB=22
    overlap=size_stepS/size_frameS
    time_stepF0=0 # 0 #value by default (0-->0.75/low_f0) Other values are in miliseconds
    low_f0=75 #75
    high_f0=600 #600
    maxPeriodVUV=0.02 #20 #milisecionds
    averagePeriodvuv=0.01#10 #milisecionds
    VUV(audio, path_base+'vuv.txt', path_base, time_stepF0, low_f0, high_f0, maxPeriodVUV, averagePeriodvuv)
    segmentsOn, fs=decode_Texgrid(path_base+'vuv.txt', audio, 'Onset')
    segmentsOff, fs=decode_Texgrid(path_base+'vuv.txt', audio, 'Offset')
    meOn, stOn=extractTrans(segmentsOn, fs, size_frameS, size_stepS, nB)
    meOff, stOff=extractTrans(segmentsOff, fs, size_frameS, size_stepS, nB)
    
#    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)
#    ax = axs[0]
#    ax.errorbar(range(25), meOn, yerr=stOn)
#    ax.set_title('Energy of Onset')
#    
#    # With 4 subplots, reduce the number of axis ticks to avoid crowding.
#   
#    ax = axs[1]
#    ax.errorbar(range(25), meOff, yerr=stOff)
#    ax.set_title('Energy of Offset')
#    plt.show()
    return meOn, stOn, meOff, stOff

    
def VUV(audio, results, path_base, time_stepF0, lowf0, highf0, maxVUVPeriod, averageVUVPeriod):
    #print 'praat '+ path_base+'vuv.praat '+audio+' '+ results+' '+str(lowf0)+' '+str(highf0)+' '+str(time_stepF0)+' '+str(maxVUVPeriod)+' '+str(averageVUVPeriod)
    os.system(path_base+'../../Toolkits/praat '+ path_base+'vuv.praat '+audio+' '+ results+' '+str(lowf0)+' '+str(highf0)+' '+str(time_stepF0)+' '+str(maxVUVPeriod)+' '+str(averageVUVPeriod))
    

def bark(f):
	x=(f*0.00076)
	x2=(f/7500)**2
	b=[]
	for i in range (0,len(f)):
		b.append(13*( math.atan(x[i]) )+3.5*( math.atan(x2[i]))) #Bark scale values
	return (b)

def barke(x,Fs, nfft=2048, nB=25):
		"""
		e: Energy in frequency bands according to the Bark scale
		x: signal 
		Fs: sampling frequency
            nfft: number of points for the Fourier transform
            nB: number of bands for energy computation
		"""
		y = fftsolp(x,nfft)
		f = (Fs/2)*(np.linspace(0,1,nfft/2+1))
		barkScale = bark(f)
		barkIndices = []
		for i in range (0,len(barkScale)):
			barkIndices.append(int(barkScale[i]))

		barkIndices = np.asarray(barkIndices)
		
		barkEnergy=[]
		for i in range (nB):
			brk = np.nonzero(barkIndices==i)
			brk = np.asarray(brk)[0]
			sizeb=len(brk)
			if (sizeb>0): 
				barkEnergy.append(sum(np.abs(y[brk]))/sizeb)
			else:
				barkEnergy.append(0)  

		e = np.asarray(barkEnergy)
		e = np.log(e)
		return e


def fftsolp(x,nfft):
     """
     STFT for compute the energy in Bark scale
     x: signal
     nffft: number of points of the Fourier transform
     """
     window = np.hamming(len(x)/4)
     noverlap = np.ceil(len(window)/2)
    
     nx = len(x)
     nwind = len(window)
    
     ncol = np.fix((nx-noverlap)/(nwind-noverlap))
     ncol = int(ncol)
     colindex = (np.arange(0,ncol))*(nwind-noverlap) 
     colindex = colindex.astype(int) 
    
     rowindex = np.arange(0,nwind) 
     rowindex = rowindex.astype(int) 
     rowindex = rowindex[np.newaxis] 
     rowindex = rowindex.T 
    
     y = np.zeros((nwind,ncol),dtype=np.int)
     d = np.ones((nwind,ncol),dtype=np.int)
    
    
     y = x[d*(rowindex+colindex)]
     window = window.astype(float) 
     window = window[np.newaxis]  
     window = window.T
     new = window*d
     y = new*y
     y = y[:,0] 
     
     y = np.fft.fft(y,nfft)
     y = (y[0:int(nfft/2+1)])
     return y

    
def decode_Texgrid(file_textgrid, file_audio, type_segm, win_trans=0.04):
    fid=open(file_textgrid)
    data=fid.read()
    fs, data_audio=read(file_audio)
    if (type_segm=='Unvoiced' or type_segm=='Onset'):   
        pos=multi_find(data, '"U"')
    elif (type_segm=='Voiced' or type_segm=='Offset'):
        pos=multi_find(data, '"V"')
    segments=[]
    for j in range(len(pos)):
        pos2=multi_find(data[0:pos[j]], '\n')
        nP=len(pos2)
        inicio=data[pos2[nP-3]:pos2[nP-2]]
        fin=data[pos2[nP-2]:pos2[nP-1]]
        inicioVal=int(float(inicio)*fs)
        finVal=int(float(fin)*fs)-1
        if (type_segm=='Onset' or type_segm=='Offset'):
            segments.append(data_audio[int(finVal-win_trans*fs):int(finVal+win_trans*fs)])
        else:
            segments.append(data_audio[inicioVal:finVal])
    return segments, fs

def extractTrans(segments, fs, size_frameS, size_stepS, nB, nfft=2048):
    frames=[]
    overlap=size_stepS/size_frameS
    for j in range(len(segments)):
        if (len(segments[j])>size_frameS):
            nF=int((len(segments[j])/size_frameS)/overlap)-1
            for iF in range(nF):
                frames.append(np.hamming(size_frameS)*segments[j][int(iF*size_stepS):int(iF*size_stepS+size_frameS)])
                    
    BarkEn=np.zeros((len(frames),nB)) 
    for j in range(len(frames)):    
        BarkEn[j,:]=(barke(frames[j],fs, nfft, nB))

    me=BarkEn.mean(0)
    st=BarkEn.std(0,ddof=1)

    return me, st

def multi_find(s, r):
    s_len = len(s)
    r_len = len(r)
    _complete = []
    if s_len < r_len:
        n = -1
    else:
        for i in range(s_len):
            # search for r in s until not enough characters are left
            if s[i:i + r_len] == r:
                _complete.append(i)
            else:
                i = i + 1
    return(_complete)


In [10]:
path_base='C:/Users\elmon/Dropbox/Parkinson/PDTool/src/artCont/'
file_features='C:/Users\elmon/Dropbox/Parkinson/PDTool/src/artCont/referenceHCmale.txt'
file_featuresMean='C:/Users\elmon/Dropbox/Parkinson/PDTool/src/artCont/referenceHCmalemean.txt'
path_audio='C:/Users/elmon/Desktop/pcgitacontHC/male/'

hf =os.listdir(path_audio)
hf.sort()
Features=[]
for j in range(len(hf)):    
    audio=path_audio+hf[j]
    ID=int(hf[j][0:3])
    task=hf[j][4:6]
    if ID>50:
        if not task.find('word')>0:
            meOn, stOn, meOff, stOff=artCont(audio, path_base)
            feat=[meOn, stOn, meOff, stOff]
            Features.append(np.hstack(feat))
            print (hf[j])
Features=np.asarray(Features)
FeaturesMean=np.nanmean(Features,0)
print(FeaturesMean)
np.savetxt(file_features, Features, fmt="%.2f")
np.savetxt(file_featuresMean, FeaturesMean, fmt="%.2f")



C:\Users\elmon\Anaconda3\lib\site-packages\ipykernel\__main__.py:54: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


052_ddk1_PCGITA.wav
052_ddk2_PCGITA.wav
052_ddk3_PCGITA.wav
052_ddk4_PCGITA.wav
052_ddk5_PCGITA.wav
052_ddk6_PCGITA.wav
052_monologue_PCGITA.wav
052_readtext_PCGITA.wav
052_sent01_PCGITA.wav
052_sent02_PCGITA.wav
052_sent03_PCGITA.wav
052_sent04_PCGITA.wav
052_sent05_PCGITA.wav
052_sent06_PCGITA.wav
052_sent07_PCGITA.wav
052_sent08_PCGITA.wav
052_sent09_PCGITA.wav
052_sent10_PCGITA.wav
054_ddk1_PCGITA.wav
054_ddk2_PCGITA.wav
054_ddk3_PCGITA.wav
054_ddk4_PCGITA.wav
054_ddk5_PCGITA.wav
054_ddk6_PCGITA.wav
054_monologue_PCGITA.wav
054_readtext_PCGITA.wav
054_sent01_PCGITA.wav
054_sent02_PCGITA.wav
054_sent03_PCGITA.wav
054_sent04_PCGITA.wav
054_sent05_PCGITA.wav
054_sent06_PCGITA.wav
054_sent07_PCGITA.wav
054_sent08_PCGITA.wav
054_sent09_PCGITA.wav
054_sent10_PCGITA.wav
069_ddk1_PCGITA.wav
069_ddk2_PCGITA.wav
069_ddk3_PCGITA.wav
069_ddk4_PCGITA.wav
069_ddk5_PCGITA.wav
069_ddk6_PCGITA.wav
069_monologue_PCGITA.wav
069_readtext_PCGITA.wav
069_sent01_PCGITA.wav
069_sent02_PCGITA.wav
069_sent0

C:\Users\elmon\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\elmon\Anaconda3\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


089_monologue_PCGITA.wav
089_readtext_PCGITA.wav
089_sent01_PCGITA.wav
089_sent02_PCGITA.wav
089_sent03_PCGITA.wav
089_sent04_PCGITA.wav
089_sent05_PCGITA.wav
089_sent06_PCGITA.wav
089_sent07_PCGITA.wav
089_sent08_PCGITA.wav
089_sent09_PCGITA.wav
089_sent10_PCGITA.wav
090_ddk1_PCGITA.wav
090_ddk2_PCGITA.wav
090_ddk3_PCGITA.wav
090_ddk4_PCGITA.wav
090_ddk5_PCGITA.wav
090_ddk6_PCGITA.wav
090_monologue_PCGITA.wav
090_readtext_PCGITA.wav
090_sent01_PCGITA.wav
090_sent02_PCGITA.wav
090_sent03_PCGITA.wav
090_sent04_PCGITA.wav
090_sent05_PCGITA.wav
090_sent06_PCGITA.wav
090_sent07_PCGITA.wav
090_sent08_PCGITA.wav
090_sent09_PCGITA.wav
090_sent10_PCGITA.wav
091_ddk1_PCGITA.wav
091_ddk2_PCGITA.wav
091_ddk3_PCGITA.wav
091_ddk4_PCGITA.wav
091_ddk5_PCGITA.wav
091_ddk6_PCGITA.wav
091_monologue_PCGITA.wav
091_readtext_PCGITA.wav
091_sent01_PCGITA.wav
091_sent02_PCGITA.wav
091_sent03_PCGITA.wav
091_sent04_PCGITA.wav
091_sent05_PCGITA.wav
091_sent06_PCGITA.wav
091_sent07_PCGITA.wav
091_sent08_PCGITA.w